<a href="https://colab.research.google.com/github/pankajagrawal925/Text-Content-Recommenders/blob/master/Text_Recommenders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendations with Document Similarity

In [1]:
!pip install textsearch
!pip install contractions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

     |████████████████████████████████| 317kB 8.2MB/s 
     |████████████████████████████████| 245kB 39.6MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81715 sha256=49748c8df1ba2d7ff515a3fe60b6df3a914e0b3bda153e7d1eccf91c38afbc40
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Load and View Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/tmdb_5000_movies (1).csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
budget                  4803 non-null int64
genres                  4803 non-null object
homepage                1712 non-null object
id                      4803 non-null int64
keywords                4803 non-null object
original_language       4803 non-null object
original_title          4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
revenue                 4803 non-null int64
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null 

In [5]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [6]:
df = df[['title', 'tagline', 'overview', 'popularity']]
df.tagline.fillna('', inplace=True)
df['description'] = df['tagline'].map(str) + ' ' + df['overview']
df.dropna(inplace=True)
df = df.sort_values(by=['popularity'], ascending=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4800 entries, 546 to 4553
Data columns (total 5 columns):
title          4800 non-null object
tagline        4800 non-null object
overview       4800 non-null object
popularity     4800 non-null float64
description    4800 non-null object
dtypes: float64(1), object(4)
memory usage: 225.0+ KB


In [7]:
df.head()

,title,tagline,overview,popularity,description
546,Minions,"Before Gru, they had a history of bad bosses","Minions Stuart, Kevin and Bob are recruited by...",875.581305,"Before Gru, they had a history of bad bosses M..."
95,Interstellar,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,724.247784,Mankind was born on Earth. It was never meant ...
788,Deadpool,Witness the beginning of a happy ending,Deadpool tells the origin story of former Spec...,514.569956,Witness the beginning of a happy ending Deadpo...
94,Guardians of the Galaxy,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",481.098624,All heroes start somewhere. Light years from E...
127,Mad Max: Fury Road,What a Lovely Day.,An apocalyptic story set in the furthest reach...,434.278564,What a Lovely Day. An apocalyptic story set in...


## Build a Movie Recommender System

In [8]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['description']))
len(norm_corpus)

4800

In [9]:
norm_corpus

array(['gru history bad bosses minions stuart kevin bob recruited scarlet overkill supervillain alongside inventor husband herb hatches plot take world',
       'mankind born earth never meant die interstellar chronicles adventures group explorers make use newly discovered wormhole surpass limitations human space travel conquer vast distances involved interstellar voyage',
       'witness beginning happy ending deadpool tells origin story former special forces operative turned mercenary wade wilson subjected rogue experiment leaves accelerated healing powers adopts alter ego deadpool armed new abilities dark twisted sense humor deadpool hunts man nearly destroyed life',
       ...,
       'one way 100 fools stand way hitchhiker named martel gordone gets fight two bikers prostitute one bikers killed gordone arrested sent prison joins prisons boxing team effort secure early parole establish dominance prisons toughest gang',
       'dare go man affair married woman dropped wrong street go

# Extract TF-IDF features

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1,2),min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(4800, 20468)

# Compute PairWise Document Similarity

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4760,4761,4762,4763,4764,4765,4766,4767,4768,4769,4770,4771,4772,4773,4774,4775,4776,4777,4778,4779,4780,4781,4782,4783,4784,4785,4786,4787,4788,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,1.00000,0.000000,0.000000,0.000000,0.006070,0.008067,0.0,0.000000,0.0,0.000000,0.025531,0.008554,0.018111,0.000000,0.000000,0.000000,0.000000,0.007439,0.010454,0.000000,0.000000,0.008190,0.008365,0.010035,0.000000,0.000000,0.050976,0.006502,0.0,0.010728,0.0,0.006908,0.000000,0.167573,0.000000,0.000000,0.000000,0.000000,0.009191,0.053475,...,0.000000,0.009711,0.006508,0.000000,0.0,0.000000,0.0,0.028409,0.0,0.000000,0.000000,0.008870,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.033246,0.0,0.000000,0.000000,0.000000,0.000000,0.034092,0.018754,0.000000,0.037924,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.009646
1,0.00000,1.000000,0.000000,0.017839,0.007967,0.000000,0.0,0.012501,0.0,0.014840,0.000000,0.000000,0.000000,0.000000,0.012814,0.000000,0.000000,0.024144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008101,0.000000,0.000000,0.016898,0.0,0.017789,0.0,0.008885,0.009432,0.000000,0.000000,0.014947,0.000000,0.000000,0.000000,0.022738,...,0.000000,0.000000,0.019783,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.011407,0.011409,0.000000,0.011632,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.015329,0.0,0.008367,0.0,0.000000,0.000000,0.021596,0.000000,0.000000,0.000000,0.000000,0.017561,0.0,0.019152,0.000000,0.000000,0.000000,0.000000,0.007963
2,0.00000,0.000000,1.000000,0.000000,0.017176,0.000000,0.0,0.000000,0.0,0.024326,0.005471,0.018038,0.000000,0.000000,0.000000,0.005099,0.004985,0.004705,0.000000,0.004843,0.000000,0.017026,0.000000,0.003672,0.003984,0.030998,0.006959,0.000000,0.0,0.006117,0.0,0.019548,0.020365,0.009213,0.028467,0.010515,0.004198,0.006311,0.015154,0.012698,...,0.020597,0.004723,0.000000,0.016673,0.0,0.000000,0.0,0.000000,0.0,0.006625,0.000000,0.000000,0.006972,0.000000,0.010574,0.0,0.008222,0.008604,0.012782,0.015353,0.006259,0.000000,0.0,0.000000,0.0,0.010555,0.000000,0.000000,0.000000,0.000000,0.000000,0.006903,0.005023,0.0,0.012893,0.000000,0.025975,0.000000,0.027126,0.009340
3,0.00000,0.017839,0.000000,1.000000,0.000000,0.022414,0.0,0.000000,0.0,0.037207,0.000000,0.000000,0.000000,0.000000,0.027958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012232,0.017893,0.043639,0.000000,0.000000,0.000000,0.0,0.023127,0.0,0.000000,0.000000,0.000000,0.000000,0.009324,0.022995,0.000000,0.000000,0.044476,...,0.000000,0.000000,0.014029,0.030561,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.056761,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.018486,0.000000,0.000000,0.000000,0.060846,0.025035,0.0,0.036237,0.030516,0.022605,0.000000,0.000000,0.000000
4,0.00607,0.007967,0.017176,0.000000,1.000000,0.004672,0.0,0.064572,0.0,0.000000,0.000000,0.036611,0.023879,0.014421,0.012316,0.019834,0.008170,0.004308,0.025459,0.000000,0.008907,0.023736,0.011341,0.003009,0.000000,0.026928,0.000000,0.010026,0.0,0.019557,0.0,0.028521,0.016018,0.011311,0.016912,0.022974,0.054142,0.013767,0.027397,0.005416,...,0.005088,0.015097,0.030759,0.013663,0.0,0.012624,0.0,0.000000,0.0,0.000000,0.027018,0.012025,0.005714,0.000000,0.017230,0.0,0.022327,0.007051,0.038014,0.011783,0.018612,0.021468,0.0,0.003767,0.0,0.008650,0.009499,0.000000,0.012749,0.000000,0.022056,0.019659,0.036850,0.0,0.015824,0.000000,0.076022,0.004515,0.043469,0.011464


# Get List of Movie Titles

In [0]:
movie_titles = df['title'].values

In [13]:
movie_titles

array(['Minions', 'Interstellar', 'Deadpool', ..., 'Penitentiary',
       'Alien Zone', 'America Is Still the Place'], dtype=object)

#Find Top Similar Movies for a Sample Movie
Let's take Minions the most popular movie in the dataframe above and try and find the most similar movies which can be recommended

In [14]:
movie_idx = np.where(movie_titles=='Minions')[0][0]
movie_idx

0

Get Movie Similarities

In [15]:
movie_sim = doc_sim_df.iloc[movie_idx].values
movie_sim

array([1.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.00964634])

**Get top 5 similar movie ids**

In [16]:
similar_movie_ids = np.argsort(-movie_sim)[1:6]
similar_movie_ids

array([ 33,  60, 737, 490, 298])

**Get top 5 similar movies**

In [17]:
sim_movies = movie_titles[similar_movie_ids]
sim_movies

array(['Despicable Me 2', 'Despicable Me',
       'Teenage Mutant Ninja Turtles: Out of the Shadows', 'Superman',
       'Rise of the Guardians'], dtype=object)

#Build a movie recommender function to recommend top 5 similar movies for any movie
The movie title, movie title list and document similarity matrix dataframe will be given as inputs to the function

In [0]:
def movie_recommender(movie_title,movies = movie_titles,doc_sims=doc_sim_df):
  movie_idx = np.where(movies==movie_title)[0][0]
  movie_sim = doc_sim_df.iloc[movie_idx].values
  movie_sim_idx = np.argsort(-movie_sim)[1:6]
  sim_movies = movies[movie_sim_idx]
  return sim_movies


# Get Popular Movie Recommendations

In [0]:
popular_movies = ['Minions', 'Interstellar', 'Deadpool', 'Jurassic World', 'Pirates of the Caribbean: The Curse of the Black Pearl',
              'Dawn of the Planet of the Apes', 'The Hunger Games: Mockingjay - Part 1', 'Terminator Genisys', 
              'Captain America: Civil War', 'The Dark Knight', 'The Martian', 'Batman v Superman: Dawn of Justice', 
              'Pulp Fiction', 'The Godfather', 'The Shawshank Redemption', 'The Lord of the Rings: The Fellowship of the Ring',  
              'Harry Potter and the Chamber of Secrets', 'Star Wars', 'The Hobbit: The Battle of the Five Armies',
              'Iron Man']

In [20]:
for movie in popular_movies:
  print("movie name:",movie)
  print("top 5 recommended movie are:", movie_recommender(movie,movies=movie_titles,doc_sims = doc_sim_df))
  print()

movie name: Minions
top 5 recommended movie are: ['Despicable Me 2' 'Despicable Me'
 'Teenage Mutant Ninja Turtles: Out of the Shadows' 'Superman'
 'Rise of the Guardians']

movie name: Interstellar
top 5 recommended movie are: ['Gattaca' 'Space Pirate Captain Harlock' 'Space Cowboys'
 'Starship Troopers' 'Final Destination 2']

movie name: Deadpool
top 5 recommended movie are: ['Silent Trigger' 'Underworld: Evolution' 'Bronson' 'Shaft' 'Don Jon']

movie name: Jurassic World
top 5 recommended movie are: ['Jurassic Park' 'The Lost World: Jurassic Park'
 "National Lampoon's Vacation" 'The Nut Job' 'Vacation']

movie name: Pirates of the Caribbean: The Curse of the Black Pearl
top 5 recommended movie are: ["Pirates of the Caribbean: Dead Man's Chest"
 'Pirates of the Caribbean: On Stranger Tides' 'The Pirate'
 'The Pirates! In an Adventure with Scientists!' 'Joyful Noise']

movie name: Dawn of the Planet of the Apes
top 5 recommended movie are: ['Battle for the Planet of the Apes' 'Groove

# Movie Recommendation with Embeddings
Here, we will use FastText model and train it on our corpus

In [0]:
from gensim.models import FastText

tokenized_docs = [doc.split() for doc in norm_corpus]
ft_model = FastText(tokenized_docs, size=300, window=30, min_count=2, workers=4, sg=1, iter=50)

#Generate document level embeddings
Word embedding models give us an embedding for each word, how can we use it for downstream ML\DL tasks? one way is to flatten it or use sequential models. A simpler approach is to average all word embeddings for words in a document and generate a fixed-length document level emebdding

In [0]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [23]:
doc_vecs_ft = averaged_word2vec_vectorizer(tokenized_docs, ft_model, 300)
doc_vecs_ft.shape

(4800, 300)

#Get Movie Recommendations
We will again use cosine similarity to generate recommendations

In [24]:
doc_sim = cosine_similarity(doc_vecs_ft)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4760,4761,4762,4763,4764,4765,4766,4767,4768,4769,4770,4771,4772,4773,4774,4775,4776,4777,4778,4779,4780,4781,4782,4783,4784,4785,4786,4787,4788,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,1.000000,0.491099,0.508440,0.499601,0.522898,0.493992,0.481054,0.439096,0.387649,0.490585,0.525403,0.558861,0.506370,0.505442,0.491140,0.492397,0.499719,0.551673,0.476119,0.488747,0.493320,0.557325,0.538231,0.522206,0.561559,0.497112,0.470204,0.526913,0.500252,0.536653,0.491296,0.540086,0.498287,0.661041,0.502511,0.549193,0.563571,0.470401,0.498773,0.567635,...,0.553580,0.585416,0.481002,0.521558,0.0,0.524586,0.436326,0.529679,0.414642,0.490994,0.498537,0.568023,0.455167,0.519819,0.431205,0.471550,0.491847,0.415932,0.479745,0.491141,0.532413,0.473514,0.346566,0.576964,0.510852,0.488103,0.518034,0.518046,0.495527,0.534322,0.552430,0.463204,0.592162,0.476162,0.509764,0.491498,0.544829,0.493601,0.514888,0.564069
1,0.491099,1.000000,0.566972,0.542050,0.602516,0.527457,0.506717,0.593457,0.474638,0.521914,0.571479,0.576482,0.524985,0.494011,0.650534,0.463969,0.531156,0.569632,0.608459,0.502285,0.470636,0.576834,0.563069,0.511981,0.569920,0.516227,0.492611,0.578808,0.597804,0.609579,0.562744,0.567985,0.533085,0.448598,0.516128,0.585825,0.592356,0.469906,0.515283,0.611261,...,0.533559,0.574644,0.636012,0.541745,0.0,0.558038,0.474878,0.563697,0.460522,0.533589,0.649861,0.587955,0.544960,0.565575,0.506611,0.437871,0.530173,0.488425,0.445410,0.509222,0.544266,0.546413,0.296354,0.531124,0.507620,0.506060,0.541641,0.575346,0.462152,0.492076,0.559364,0.480571,0.538415,0.486793,0.567375,0.493636,0.478263,0.487538,0.531165,0.571149
2,0.508440,0.566972,1.000000,0.577382,0.591965,0.462826,0.518174,0.514529,0.512972,0.593316,0.584979,0.552814,0.514746,0.518586,0.581510,0.510964,0.563026,0.556175,0.532405,0.526444,0.471353,0.543567,0.569837,0.502398,0.558650,0.560688,0.462006,0.538247,0.543847,0.544265,0.573038,0.538941,0.541222,0.516610,0.580441,0.564407,0.546123,0.492250,0.581416,0.576635,...,0.575809,0.552077,0.542923,0.498978,0.0,0.520084,0.492428,0.549547,0.440792,0.553202,0.526624,0.584852,0.537191,0.553172,0.483682,0.434403,0.501088,0.501408,0.470502,0.473306,0.527024,0.530522,0.313938,0.542157,0.456073,0.465562,0.523666,0.541915,0.534617,0.528953,0.534462,0.537058,0.560981,0.487341,0.568207,0.508747,0.542787,0.539310,0.527676,0.542354
3,0.499601,0.542050,0.577382,1.000000,0.593465,0.516280,0.464132,0.514896,0.498781,0.542706,0.561492,0.539889,0.506633,0.511917,0.574582,0.557641,0.553136,0.578178,0.519743,0.484349,0.477436,0.540883,0.558823,0.551784,0.590326,0.516627,0.426192,0.500219,0.483160,0.610745,0.505829,0.511828,0.512491,0.494239,0.501811,0.591790,0.598400,0.482348,0.544636,0.578893,...,0.498521,0.491580,0.528282,0.560581,0.0,0.487996,0.399945,0.552194,0.388544,0.509592,0.488317,0.541869,0.464953,0.521835,0.541039,0.447756,0.525197,0.488180,0.463142,0.553774,0.564078,0.497540,0.340004,0.552841,0.501622,0.484853,0.546586,0.589722,0.470125,0.520741,0.531434,0.574895,0.577436,0.493903,0.539647,0.533235,0.562266,0.499612,0.496677,0.540088
4,0.522898,0.602516,0.591965,0.593465,1.000000,0.570414,0.569773,0.668487,0.559169,0.545415,0.595153,0.626347,0.553360,0.559663,0.662991,0.568405,0.647197,0.624332,0.632446,0.582372,0.540621,0.611019,0.634381,0.601113,0.595807,0.511322,0.488727,0.625997,0.601200,0.673677,0.570284,0.619832,0.631452,0.558328,0.626759,0.614827,0.644804,0.548273,0.635957,0.606338,...,0.591246,0.566969,0.647805,0.584320,0.0,0.577198,0.534760,0.587189,0.434573,0.540593,0.621390,0.605360,0.566392,0.600637,0.583164,0.531561,0.584597,0.552837,0.572490,0.600195,0.587768,0.574146,0.334447,0.601425,0.570819,0.551264,0.624660,0.569082,0.590939,0.549015,0.617179,0.627095,0.645846,0.464449,0.605486,0.537725,0.644473,0.569277,0.631921,0.634419


In [26]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movie_titles, doc_sims=doc_sim_df))
    print()

Movie: Minions
Top 5 recommended Movies: ['Despicable Me' 'Time Bandits'
 'Rise of the Entrepreneur: The Search for a Better Way'
 'Austin Powers: The Spy Who Shagged Me' 'Despicable Me 2']

Movie: Interstellar
Top 5 recommended Movies: ['Prometheus' 'Gattaca' 'Space Cowboys'
 'Sea Rex 3D: Journey to a Prehistoric World' 'Jason X']

Movie: Deadpool
Top 5 recommended Movies: ['Fantastic Four' 'Banshee Chapter' 'The Hunted' 'Enough' 'Spider-Man 3']

Movie: Jurassic World
Top 5 recommended Movies: ['Jurassic Park' 'The Lost World: Jurassic Park' 'Jurassic Park III'
 "National Lampoon's Vacation" 'Walking With Dinosaurs']

Movie: Pirates of the Caribbean: The Curse of the Black Pearl
Top 5 recommended Movies: ['Pirates of the Caribbean: On Stranger Tides'
 'The Pirates! In an Adventure with Scientists!'
 "Pirates of the Caribbean: Dead Man's Chest"
 'American Ninja 2: The Confrontation' 'In the Name of the King III']

Movie: Dawn of the Planet of the Apes
Top 5 recommended Movies: ['Battle